In [1]:
%%capture
!pip install fairlearn

In [2]:
# General imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# General library for dataset creation
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# ML models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Fairlearn algorithms and utils
from fairlearn.postprocessing import ThresholdOptimizer
# from fairlearn.reductions import GridSearch, EqualizedOdds

# Metrics
from fairlearn.metrics import (
    MetricFrame,
    selection_rate, demographic_parity_difference, demographic_parity_ratio,
    false_positive_rate, false_negative_rate,
    false_positive_rate_difference, false_negative_rate_difference,
    equalized_odds_difference, equalized_odds_ratio)
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import mutual_info_score, adjusted_mutual_info_score, normalized_mutual_info_score

# Create Dataset

In [3]:
def create_synth(dim=15000, l_y=4, l_m_y=0, thr_supp=1, l_h_r=1.5,  l_h_q=1, l_m=1, p_u=1, l_r=False, l_o=False, l_y_b=0, l_q=2, sy=5, l_r_q=0):
    """Generate a synthetic dataset.

    Parameters
    ----------
    dim : int
        Dimension of the dataset
    l_y : float, optional
        Lambda coefficient for historical bias on the target y
    l_m_y : float, optional
        Lambda coefficient for measurement bias on the target y
    thr_supp: float, optional
        Threshold correlation for discarding features too much correlated with s
    l_h_r: float, optional
        Lambda coefficient for historical bias on R
    l_h_q: float, optional
        Lambda coefficient for historical bias on Q
    l_m: float, optional
        Lambda coefficient for measurement bias. If l_m!=0 P substitute R.
    p_u: float, optional
        Percentage of undersampling instance with A=1
    l_r: bool, optional
        Boolean for inducing representation bias, that is undersampling conditioning on a variable, e.g. X2
    l_o: bool, optional
        Boolean variable for excluding an important variable, e.g. X2
    l_y_b: float, optional
        Lambda coefficient for interaction proxy bias
    l_q: float, optional
        Lambda coefficient for importance of Q for Y
    sy: float, optional
        Standard deviation of the noise of Y
    l_r_q: float, optional
        Lambda coefficient that quantifies the influence from R to Q

    Returns
    -------
    list
        a list datasets train and test for: complete dataset, individual and suppression.
    """
    np.random.seed(42)
    # # # dataset
    # Random var
    N1 = np.random.gamma(2, 3, dim)
    N2 = np.random.normal(1, 0.5, dim)
    Np = np.random.normal(0, 2, dim)
    Ny = np.random.normal(0, sy, dim)
    A = np.random.binomial(1, 0.5, size=dim)

    # X var
    # Variable R defined as the salary
    R = N1 - l_h_r*A 
    # Variable R influence Q, a discrete variable that define a zone in a city
    R_A = 1/(1 + np.exp(l_r_q*R - l_h_q*A))
    Q = np.random.binomial(3, R_A)

    # Y var
    # y target, with measurement and historical bias
    y = R - l_q*Q - l_y*A - l_m_y*A + Ny + l_y_b*R*A
    # y only historical, no measurement bias
    y_real = R - l_q*Q - l_y*A + Ny + l_y_b*R*A

    if l_m!=0:
      # Proxy for R, e.g. the dimension of the house
      P = R - A*l_m + Np
      print("Correlation between R and P: ", np.corrcoef(P, R))
      dtf = pd.DataFrame({'P': P, 'Q':Q, 'A':A, 'Y':y, 'Y_real':y_real})
    else: 
      dtf = pd.DataFrame({'R':R, 'Q':Q, 'A':A, 'Y':y, 'Y_real':y_real})

    # Udersample
    int_p_u = int(((dtf['A']==1).sum())*p_u)
    if int_p_u > 0:
      if l_r:
          # Undersample with increasing R, for A=1 the people will results poor 
          drop_index = dtf.loc[dtf['A']==1, :].sort_values(by='R', ascending=True).index
          dtf = dtf.drop(drop_index[int_p_u:])
      else:
          dtf = dtf.drop(dtf.index[dtf['A']==1][int_p_u:])
    # Delete an important variable for omission: R or P
    if l_o:
      if 'R' in dtf.columns:
        del dtf['R']
      elif 'P' in dtf.columns:
        del dtf['P']
      else:
        print("Condition non possible. How I could get here?")
    # Define feature matrix X and target Y
    X = dtf.reset_index(drop=True)
    y = X['Y']
    y_real = X['Y_real']
    del X['Y']
    del X['Y_real']
    # Define threshold making y binary
    thres = y.mean()
    y = pd.Series(1*(y>thres))
    y_real = pd.Series(1*(y_real>thres))
    # Split train test
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42, stratify=X['A']==1)
    # individual set
    X_ind_train = X_train[[i for i in X_train.columns if i!='A']]
    X_ind_test = X_test[[i for i in X_test.columns if i!='A']]
    # suppression set
    X_supp_train = X_train[[i for i in X_train.columns if i!='A' and 
                            abs(np.corrcoef(X_train[i], X_train['A'])[0,1])<thr_supp]]
    X_supp_test = X_test[[i for i in X_test.columns if i!='A' and 
                          abs(np.corrcoef(X_train[i], X_train['A'])[0,1])<thr_supp]]
    # get y_test not biased
    y_train_real = y_real[y_train.index]
    y_test_real = y_real[y_test.index]

    return X_train, X_ind_train, X_supp_train, X_test, X_ind_test, X_supp_test, y_train, y_test, y_train_real, y_test_real

# Fit model

In [4]:
def fit_models(X_train, X_ind_train, X_supp_train, y_train):
  # Fit classifier
  model = RandomForestClassifier
  clf = model(random_state=0, max_depth=5).fit(X_train, y_train)
  clf_ind = model(random_state=0, max_depth=5).fit(X_ind_train, y_train)
  clf_supp = model(random_state=0, max_depth=5).fit(X_supp_train, y_train)

  # KNN Individual metric
  neigh = KNeighborsClassifier(n_neighbors=5, weights='distance')
  neigh.fit(X_ind_train, y_train)
  neigh_supp = KNeighborsClassifier(n_neighbors=5, weights='distance')
  neigh_supp.fit(X_supp_train, y_train)

  return clf, clf_ind, clf_supp, neigh, neigh_supp

# Fairlearn


In [5]:
# Code modified from https://github.com/fairlearn/fairlearn/blob/main/notebooks/Binary%20Classification%20with%20the%20UCI%20Credit-card%20Default%20Dataset.ipynb
# Helper functions 
def get_metrics_df(models_dict, y_true, group, X_ind_test=None, X_supp_test=None, dct_flip=None):
    metrics_dict = {
        # "Overall selection rate": (
        #     lambda x: selection_rate(y_true, x), True),
        "Demographic parity difference": (
            lambda x: demographic_parity_difference(y_true, x, sensitive_features=group), True),
        "Demographic parity ratio": (
            lambda x: demographic_parity_ratio(y_true, x, sensitive_features=group), True),
        "------": (lambda x: "", True),
        # "Overall balanced error rate": (
        #     lambda x: 1-balanced_accuracy_score(y_true, x), True),
        # "Balanced error rate difference": (
        #     lambda x: MetricFrame(metrics=balanced_accuracy_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), True),
        # " ------": (lambda x: "", True),
        # "False positive rate difference": (
        #     lambda x: false_positive_rate_difference(y_true, x, sensitive_features=group), True),
        # "False negative rate difference": (
        #     lambda x: false_negative_rate_difference(y_true, x, sensitive_features=group), True),
        "Equalized odds ratio": (
            lambda x: equalized_odds_ratio(y_true, x, sensitive_features=group), True),
        "Equalized odds difference": (
            lambda x: equalized_odds_difference(y_true, x, sensitive_features=group), True),
        "  ------": (lambda x: "", True),
        # "Overall AUC": (
        #     lambda x: roc_auc_score(y_true, x), False),
        "ACC score": (
            lambda x: accuracy_score(y_true, 1*(x>0.5)), False),
        "F1 score": (
            lambda x: f1_score(y_true, 1*(x>0.5)), False),
        "Precision": (
            lambda x: precision_score(y_true, 1*(x>0.5)), False),
        "Recall": (
            lambda x: recall_score(y_true, 1*(x>0.5)), False),
        # "AUC difference": (
        #     lambda x: MetricFrame(metrics=roc_auc_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), False),
        "ACC score difference": (
            lambda x: MetricFrame(metrics=accuracy_score, y_true=y_true, y_pred=1*(x>0.5), sensitive_features=group).difference(method='between_groups'), False),
        "   ------": (lambda x: "", True),
        "F1 score difference": (
            lambda x: MetricFrame(metrics=f1_score, y_true=y_true, y_pred=1*(x>0.5), sensitive_features=group).difference(method='between_groups'), False),
        "   ------": (lambda x: "", True),
        "Flip": (lambda x: dct_flip[x], 'no'),
        # "Individuality": (lambda x: neigh.score(X_ind_test, x), True),
        # "Individuality suppression": (lambda x: neigh_supp.score(X_supp_test, x), True),
        "    ------": (lambda x: "", True),
        "Mutual information A, y_pred": (lambda x: normalized_mutual_info_score(group, x), True),
        "Mutual information A, y_true": (lambda x: normalized_mutual_info_score(group, y_true), True),
    }
    df_dict = {}
    for metric_name, (metric_func, use_preds) in metrics_dict.items():
            list_tmp = []
            for model_name, (preds, scores) in models_dict.items():
                try:
                    if use_preds==True:
                        list_tmp.append(metric_func(preds))
                    elif use_preds==False:
                        list_tmp.append(metric_func(scores))
                    else:
                        list_tmp.append(metric_func(model_name))
                except:
                    print('problem in: ', metric_name)
                    list_tmp.append('NaN')
            df_dict[metric_name] = list_tmp
    return pd.DataFrame.from_dict(df_dict, orient="index", columns=models_dict.keys())

### Mitigation

In [6]:
def mitigations(X_train, X_test, X_ind_test, X_supp_test, y_train, y_test, y_test_real,
                thr_supp, sens_var, cond_var,
                clf, clf_ind, clf_supp
                ):
  """Method to mitigate in postprocessing the predictions of the models.

  Parameters
  ----------
  X_train : pd.DataFrame
      Training set
  X_test : pd.DataFrame
      Test set
  X_ind_test : pd.DataFrame
      Individual test set (without A)
  X_supp_test : pd.DataFrame
      Suppression test set (without varible correlated with A)
  y_train : pd.Series
      Target train
  y_test : pd.Series
      Target test
  y_test_real : pd.Series
      Target test - no measurement bias
  thr_supp: float
    Threshold for suppression method
  sens_var : str
      Name of the sensitive variable. E.g. sens_var = 'A'
  cond_var : str
      Name of the variable to condition used by the Conditional Demographic Parity.
      E.g. cond_var = 'Q'
  clf: sklearn.model
      Classifier model
  clf_ind: sklearn.model
      Classifier model for individual dataset
  clf_supp: sklearn.model
      Classifier model for suppression dataset

  Returns
  -------
  pd.DataFrame
      a DataFrame of the output results: metrics per each model.
  """
  
  # Define a dataset with sensitive/protected attribute flipped
  X_flip = X_test.copy()
  X_flip[sens_var] = 1-X_flip[sens_var]
  dct_flip={'FTU':1, 'Suppression_'+str(thr_supp):1}

  # # # unmitigated
  dct_flip['Unmitigated'] = 1-abs(clf.predict(X_test) - clf.predict(X_flip)).mean()

  # # # equalized_odds
  postprocess_est_eo = ThresholdOptimizer(estimator=clf, 
      constraints="equalized_odds", predict_method='predict_proba', prefit=True)
  postprocess_est_eo.fit(X_train, y_train, sensitive_features=X_train[sens_var])
  postprocess_preds_eo = postprocess_est_eo.predict(X_test, sensitive_features=X_test[sens_var])
  postprocess_preds_eo_flip = postprocess_est_eo.predict(X_flip, sensitive_features=X_flip[sens_var])
  dct_flip['EO'] = 1-abs(postprocess_preds_eo-postprocess_preds_eo_flip).mean()
  # # # demographic_parity
  postprocess_est_dp = ThresholdOptimizer(estimator=clf, 
    constraints="demographic_parity", predict_method='predict_proba', prefit=True)
  postprocess_est_dp.fit(X_train, y_train, sensitive_features=X_train[sens_var])
  postprocess_preds_dp = postprocess_est_dp.predict(X_test, sensitive_features=X_test[sens_var])
  postprocess_preds_dp_flip = postprocess_est_dp.predict(X_flip, sensitive_features=X_flip[sens_var])
  dct_flip['DP'] = 1-abs(postprocess_preds_dp-postprocess_preds_dp_flip).mean()
  # # # Conditional demographic_parity
  try:
    postprocess_preds_cdp = 0*postprocess_preds_dp.copy()
    postprocess_preds_cdp_flip = 0*postprocess_preds_dp.copy()
    for i in X_train[cond_var].unique():
        postprocess_est_dp_0 = ThresholdOptimizer(estimator=clf, 
          constraints="demographic_parity", predict_method='predict_proba', prefit=True)
        postprocess_est_dp_0.fit(X_train.loc[X_train[cond_var]==i], y_train[X_train[cond_var]==i], 
                                sensitive_features=X_train.loc[X_train[cond_var]==i, sens_var])
        postprocess_preds_dp_0 = postprocess_est_dp_0.predict(X_test.loc[X_test[cond_var]==i],
                                  sensitive_features=X_test.loc[X_test[cond_var]==i, sens_var])
        postprocess_preds_dp_0_flip = postprocess_est_dp_0.predict(X_flip.loc[X_flip[cond_var]==i],
                                  sensitive_features=X_flip.loc[X_flip[cond_var]==i, sens_var])
        # Define the prediction with the same dimensionality of test prediction
        postprocess_preds_cdp[X_test[cond_var]==i] = postprocess_preds_dp_0
        postprocess_preds_cdp_flip[X_test[cond_var]==i] = postprocess_preds_dp_0_flip
    dct_flip['CDP'] = 1-abs(postprocess_preds_cdp-postprocess_preds_cdp_flip).mean()
  except:
    print('Can t mitigate with CDP.')
    postprocess_preds_cdp = np.nan
    dct_flip['CDP'] = np.nan

  # # # Build dictionary of mitigated prediction
  models_dict = {"Y": (y_test, y_test),
                "Y True": (y_test_real, y_test_real),
                "Unmitigated": (clf.predict(X_test), clf.predict_proba(X_test)[:, 1]),
               "FTU":  (clf_ind.predict(X_ind_test), clf_ind.predict_proba(X_ind_test)[:, 1]),
               "Suppression_"+str(thr_supp): (clf_supp.predict(X_supp_test), clf_supp.predict_proba(X_supp_test)[:, 1]),
              "EO": (postprocess_preds_eo, postprocess_preds_eo),
               "DP": (postprocess_preds_dp, postprocess_preds_dp),
               "CDP": (postprocess_preds_cdp, postprocess_preds_cdp)}

  return models_dict, dct_flip

In [7]:
def pipeline(param_dict, sens_var = 'A', cond_var = 'Q', y_bias_meas = False):
  """Pipeline that create a synthetic dataset, fit the models, mitigate those
   and output the results metrics

  Parameters
  ----------
  param_dict : dict
      Dictonary for setting the dataset creation
  sens_var : str, optional
      Name of the sensitive variable
  cond_var : str, optional
      Name of the variable to condition used by the Conditional Demographic Parity
  y_bias_meas: bool, optional
      If true the metrics are tested on the target y without measurement bias

  Returns
  -------
  pd.DataFrame
      a DataFrame of the output results: metrics per each model.
  """
  print("The parameters for data generetion are: ", param_dict, '\n')
  thr_supp = param_dict["thr_supp"]
  # Create dataset
  print("Start creation dataset.", '\n')
  X_train, X_ind_train, X_supp_train, X_test, X_ind_test, X_supp_test, y_train, y_test, y_train_real, y_test_real = create_synth(**param_dict)
  df_total = X_train.copy()
  df_total['Y'] = y_train
  if y_bias_meas:
    df_total['Y_real'] = y_train_real
  print("The correlation matrix is: ", '\n', df_total.corr(), '\n',
        "The value counts is: ", '\n', df_total[sens_var].value_counts(), '\n')
  # Fit models
  print("Fitting models.", '\n')
  clf, clf_ind, clf_supp, neigh, neigh_supp = fit_models(X_train, X_ind_train, X_supp_train, y_train)
  # Mitigate models
  print("Mitigate models.", '\n')
  models_dict, dct_flip = \
    mitigations(X_train, X_test, X_ind_test, X_supp_test, y_train, y_test, y_test_real,
                thr_supp, sens_var, cond_var,
                clf, clf_ind, clf_supp)
  # return summary table
  print("Report output.", '\n')
  if y_bias_meas:
    return get_metrics_df(models_dict, y_test_real, X_test[sens_var], X_ind_test, X_supp_test, dct_flip)
  else:
    return get_metrics_df(models_dict, y_test, X_test[sens_var], X_ind_test, X_supp_test, dct_flip)

In [8]:
# # Synthetic example output
# thr_supp = 0.5
# param_dict = {"dim":15000, "l_y":4, "thr_supp":thr_supp, "l_h":1.5, "l_m":True,
#               "p_u":0.2, "l_r":False, "l_o":True}
# pipeline(param_dict)

Scenarios parameters
    ----------
    Parameters
    ----------
    dim : int
        Dimension of the dataset
    l_y : float, optional
        Lambda coefficient for historical bias on the target y
    l_m_y : float, optional
        Lambda coefficient for measurement bias on the target y
    thr_supp: float, optional
        Threshold correlation for discarding features too much correlated with s
    l_h_r: float, optional
        Lambda coefficient for historical bias on R
    l_h_q: float, optional
        Lambda coefficient for historical bias on Q
    l_m: float, optional
        Lambda coefficient for measurement bias. If l_m!=0 P substitute R.
    p_u: float, optional
        Percentage of undersampling instance with A=1
    l_r: bool, optional
        Boolean for inducing representation bias, that is undersampling conditioning on a variable, e.g. X2
    l_o: bool, optional
        Boolean variable for excluding an important variable, e.g. X2
    l_y_b: float, optional
        Lambda coefficient for interaction proxy bias
    l_q: float, optional
        Lambda coefficient for importance of Q for Y
    sy: float, optional
        Standard deviation of the noise of Y
    l_r_q: float, optional
        Lambda coefficient that quantifies the influence from R to Q

# Scenarios

## S1 - no historical bias

In [9]:
# no additional bias
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":0,  "l_h_q":0,
              "l_m":0, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 0, 'l_h_q': 0, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000 -0.002790 -0.001676  0.635310
Q -0.002790  1.000000  0.001225 -0.298573
A -0.001676  0.001225  1.000000  0.002482
Y  0.635310 -0.298573  0.002482  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,0.210188,0.210188,0.094284,0.075239,0.038963
ACC score,100.0,100.0,86.372727,86.421212,86.384848
ACC score difference,0.0,0.0,0.134439,0.146577,0.110201


In [10]:
# measurment on R
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":0,  "l_h_q":0,
              "l_m":1, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 0, 'l_h_q': 0, 'l_m': 1, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

Correlation between R and P:  [[1.         0.89928525]
 [0.89928525 1.        ]]
The correlation matrix is:  
           P         Q         A         Y
P  1.000000  0.000310 -0.108182  0.570987
Q  0.000310  1.000000  0.001225 -0.298573
A -0.108182  0.001225  1.000000  0.002482
Y  0.570987 -0.298573  0.002482  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,0.210188,0.210188,0.77221,7.414714,0.542702
ACC score,100.0,100.0,81.160606,81.057576,81.151515
ACC score difference,0.0,0.0,0.484058,0.714324,0.502237


In [11]:
# omission
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":0,  "l_h_q":0,
              "l_m":0, "p_u":1, "l_r":False, "l_o":True, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 0, 'l_h_q': 0, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': True, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           Q         A         Y
Q  1.000000  0.001225 -0.298573
A  0.001225  1.000000  0.002482
Y -0.298573  0.002482  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,0.210188,0.210188,1.163635,1.163635,0.951568
ACC score,100.0,100.0,62.766667,62.766667,62.751515
ACC score difference,0.0,0.0,0.368279,0.368279,0.410698


In [12]:
# sample
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":0,  "l_h_q":0,
              "l_m":0, "p_u":0.1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 0, 'l_h_q': 0, 'l_m': 0, 'p_u': 0.1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000 -0.007685 -0.001163  0.635270
Q -0.007685  1.000000 -0.002525 -0.306182
A -0.001163 -0.002525  1.000000 -0.000779
Y  0.635270 -0.306182 -0.000779  1.000000 
 The value counts is:  
 0    33493
1     3350
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,1.940033,1.940033,0.703916,0.57056,0.35842
ACC score,100.0,100.0,86.268459,86.296011,86.307031
ACC score difference,0.0,0.0,1.961994,1.792298,2.004423


In [13]:
# measurement bias on Y (P_Y as target). Performance are calculated on Y
param_dict = {"dim":100000, "l_y":0, "l_m_y":1.5, "thr_supp":1, "l_h_r":0,  "l_h_q":0,
              "l_m":0, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict, y_bias_meas=True)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 1.5, 'thr_supp': 1, 'l_h_r': 0, 'l_h_q': 0, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
                R         Q         A         Y    Y_real
R       1.000000 -0.002790 -0.001676  0.628497  0.603973
Q      -0.002790  1.000000  0.001225 -0.291867 -0.313287
A      -0.001676  0.001225  1.000000 -0.122700  0.002057
Y       0.628497 -0.291867 -0.122700  1.000000  0.882952
Y_real  0.603973 -0.313287  0.002057  0.882952  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,12.943638,0.194441,11.22728,0.105574,0.027654
ACC score,93.624242,100.0,83.342424,84.457576,84.324242
ACC score difference,12.749197,0.0,4.012125,0.048894,0.012482


In [14]:
# representation bias
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":0,  "l_h_q":0,
              "l_m":0, "p_u":0.1, "l_r":True, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 0, 'l_h_q': 0, 'l_m': 0, 'p_u': 0.1, 'l_r': True, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000 -0.005347 -0.332019  0.639318
Q -0.005347  1.000000  0.001442 -0.299489
A -0.332019  0.001442  1.000000 -0.250970
Y  0.639318 -0.299489 -0.250970  1.000000 
 The value counts is:  
 0    33493
1     3350
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Can t mitigate with CDP.
Report output. 

problem in:  Demographic parity difference
problem in:  Demographic parity ratio
problem in:  Equalized odds ratio
problem in:  Equalized odds difference
problem in:  ACC score
problem in:  F1 score
problem in:  Precision
problem in:  Recall
problem in:  ACC score difference
problem in:  F1 score difference
problem in:  Flip
problem in:  Flip
problem in:  Mutual inform

,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,44.187733,44.187733,45.205494,45.429749,0.587281
ACC score,100.0,100.0,86.7203,86.747851,83.41415
ACC score difference,0.0,0.0,9.407774,9.510802,26.022519


## S2 - historical bias compensation

In [15]:
# no additional bias
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":-1,
              "l_m":0, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': -1, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000  0.065214 -0.174944  0.633145
Q  0.065214  1.000000 -0.391901 -0.252705
A -0.174944 -0.391901  1.000000 -0.008692
Y  0.633145 -0.252705 -0.008692  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,1.38592,1.38592,0.123829,0.833859,0.343013
ACC score,100.0,100.0,86.342424,86.451515,86.387879
ACC score difference,0.0,0.0,0.049579,0.073861,0.017071


In [16]:
# measurment on R
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":-1,
              "l_m":1, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': -1, 'l_m': 1, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

Correlation between R and P:  [[1.         0.90538424]
 [0.90538424 1.        ]]
The correlation matrix is:  
           P         Q         A         Y
P  1.000000  0.100867 -0.259059  0.562825
Q  0.100867  1.000000 -0.391901 -0.252705
A -0.259059 -0.391901  1.000000 -0.008692
Y  0.562825 -0.252705 -0.008692  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,1.38592,1.38592,2.166053,5.590341,0.942543
ACC score,100.0,100.0,81.118182,81.090909,81.066667
ACC score difference,0.0,0.0,0.399195,0.708276,0.308267


In [17]:
# omission
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":-1,
              "l_m":0, "p_u":1, "l_r":False, "l_o":True, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': -1, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': True, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           Q         A         Y
Q  1.000000 -0.391901 -0.252705
A -0.391901  1.000000 -0.008692
Y -0.252705 -0.008692  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,1.38592,1.38592,9.791949,27.093546,0.937277
ACC score,100.0,100.0,62.7,61.587879,62.354545
ACC score difference,0.0,0.0,0.659164,1.565483,0.01975


In [18]:
# sample
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":-1,
              "l_m":0, "p_u":0.1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': -1, 'l_m': 0, 'p_u': 0.1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000  0.016263 -0.101782  0.632553
Q  0.016263  1.000000 -0.227652 -0.294001
A -0.101782 -0.227652  1.000000 -0.003284
Y  0.632553 -0.294001 -0.003284  1.000000 
 The value counts is:  
 0    33493
1     3350
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,0.952145,0.952145,1.611251,0.726413,0.588721
ACC score,100.0,100.0,86.27948,86.2905,86.362134
ACC score difference,0.0,0.0,1.37411,1.519567,1.19836


In [19]:
# measurement bias on Y (P_Y as target). Performance are calculated on Y
param_dict = {"dim":100000, "l_y":0, "l_m_y":1.5, "thr_supp":1, "l_h_r":1.5,  "l_h_q":-1,
              "l_m":0, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict, y_bias_meas=True)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 1.5, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': -1, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
                R         Q         A         Y    Y_real
R       1.000000  0.065214 -0.174944  0.645688  0.602124
Q       0.065214  1.000000 -0.391901 -0.197834 -0.266431
A      -0.174944 -0.391901  1.000000 -0.132825 -0.005467
Y       0.645688 -0.197834 -0.132825  1.000000  0.880709
Y_real  0.602124 -0.266431 -0.005467  0.880709  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,14.307192,1.291376,11.512405,3.8701,0.439987
ACC score,93.490909,100.0,83.693939,84.2,84.593939
ACC score difference,13.015815,0.0,3.381949,0.721527,0.466208


In [20]:
# representation bias
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":-1,
              "l_m":0, "p_u":0.1, "l_r":True, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': -1, 'l_m': 0, 'p_u': 0.1, 'l_r': True, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000  0.088878 -0.416227  0.645537
Q  0.088878  1.000000 -0.224779 -0.227457
A -0.416227 -0.224779  1.000000 -0.263621
Y  0.645537 -0.227457 -0.263621  1.000000 
 The value counts is:  
 0    33493
1     3350
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Can t mitigate with CDP.
Report output. 

problem in:  Demographic parity difference
problem in:  Demographic parity ratio
problem in:  Equalized odds ratio
problem in:  Equalized odds difference
problem in:  ACC score
problem in:  F1 score
problem in:  Precision
problem in:  Recall
problem in:  ACC score difference
problem in:  F1 score difference
problem in:  Flip
problem in:  Flip
problem in:  Mutual inf

,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,45.721075,45.721075,45.272154,45.43581,1.843118
ACC score,100.0,100.0,86.819484,86.847035,83.144148
ACC score difference,0.0,0.0,11.032022,11.001716,29.39222


## S3 - Historical bias on R

In [21]:
# no additional bias
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":0,
              "l_m":0, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': 0, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000 -0.002959 -0.174944  0.640068
Q -0.002959  1.000000  0.001225 -0.291867
A -0.174944  0.001225  1.000000 -0.122700
Y  0.640068 -0.291867 -0.122700  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,12.943638,12.943638,12.560752,12.948759,0.015566
ACC score,100.0,100.0,86.657576,86.621212,85.542424
ACC score difference,0.0,0.0,2.168488,2.095774,2.253742


In [22]:
# measurment on R
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":0,
              "l_m":1, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': 0, 'l_m': 1, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

Correlation between R and P:  [[1.         0.90538424]
 [0.90538424 1.        ]]
The correlation matrix is:  
           P         Q         A         Y
P  1.000000  0.000113 -0.259059  0.580852
Q  0.000113  1.000000  0.001225 -0.291867
A -0.259059  0.001225  1.000000 -0.122700
Y  0.580852 -0.291867 -0.122700  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,12.943638,12.943638,11.814556,19.069283,0.687892
ACC score,100.0,100.0,81.484848,81.360606,80.627273
ACC score difference,0.0,0.0,2.491581,2.182536,2.340378


In [23]:
# omission
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":0,
              "l_m":0, "p_u":1, "l_r":False, "l_o":True, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': 0, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': True, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           Q         A         Y
Q  1.000000  0.001225 -0.291867
A  0.001225  1.000000 -0.122700
Y -0.291867 -0.122700  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,12.943638,12.943638,36.690088,1.163635,0.951568
ACC score,100.0,100.0,64.548485,62.624242,62.615152
ACC score difference,0.0,0.0,1.291679,2.556106,2.562163


In [24]:
# sample
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":0,
              "l_m":0, "p_u":0.1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': 0, 'l_m': 0, 'p_u': 0.1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000 -0.007392 -0.101782  0.635198
Q -0.007392  1.000000 -0.002525 -0.305691
A -0.101782 -0.002525  1.000000 -0.066942
Y  0.635198 -0.305691 -0.066942  1.000000 
 The value counts is:  
 0    33493
1     3350
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,9.908602,9.908602,11.199183,10.253739,0.047646
ACC score,100.0,100.0,86.362134,86.345603,86.180295
ACC score difference,0.0,0.0,0.868329,0.953181,1.265005


In [25]:
# measurement bias on Y (P_Y as target). Performance are calculated on Y
param_dict = {"dim":100000, "l_y":0, "l_m_y":1.5, "thr_supp":1, "l_h_r":1.5,  "l_h_q":0,
              "l_m":0, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict, y_bias_meas=True)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 1.5, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': 0, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
                R         Q         A         Y    Y_real
R       1.000000 -0.002959 -0.174944  0.637131  0.610068
Q      -0.002959  1.000000  0.001225 -0.283430 -0.307326
A      -0.174944  0.001225  1.000000 -0.244939 -0.128151
Y       0.637131 -0.283430 -0.244939  1.000000  0.890457
Y_real  0.610068 -0.307326 -0.128151  0.890457  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,24.798318,13.188308,25.064487,13.131121,0.129787
ACC score,94.193939,100.0,84.036364,84.863636,83.681818
ACC score difference,11.61001,0.0,1.691165,1.502474,4.896843


In [26]:
# representation bias
param_dict = {"dim":100000, "l_y":0, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":0,
              "l_m":0, "p_u":0.1, "l_r":True, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 0, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': 0, 'l_m': 0, 'p_u': 0.1, 'l_r': True, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000 -0.005293 -0.416227  0.645785
Q -0.005293  1.000000  0.001442 -0.291786
A -0.416227  0.001442  1.000000 -0.277194
Y  0.645785 -0.291786 -0.277194  1.000000 
 The value counts is:  
 0    33493
1     3350
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Can t mitigate with CDP.
Report output. 

problem in:  Demographic parity difference
problem in:  Demographic parity ratio
problem in:  Equalized odds ratio
problem in:  Equalized odds difference
problem in:  ACC score
problem in:  F1 score
problem in:  Precision
problem in:  Recall
problem in:  ACC score difference
problem in:  F1 score difference
problem in:  Flip
problem in:  Flip
problem in:  Mutual info

,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,48.612122,48.612122,45.575221,46.690508,1.538714
ACC score,100.0,100.0,86.676218,86.841525,82.868636
ACC score difference,0.0,0.0,13.122972,12.941132,27.422468


## S4 - Historical bias on Y

In [27]:
# no additional bias
param_dict = {"dim":100000, "l_y":1.5, "l_m_y":0, "thr_supp":1, "l_h_r":0,  "l_h_q":0,
              "l_m":0, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 1.5, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 0, 'l_h_q': 0, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000 -0.002790 -0.001676  0.628497
Q -0.002790  1.000000  0.001225 -0.291867
A -0.001676  0.001225  1.000000 -0.122700
Y  0.628497 -0.291867 -0.122700  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,12.943638,12.943638,11.22728,0.105574,0.027654
ACC score,100.0,100.0,86.5,85.548485,85.518182
ACC score difference,0.0,0.0,2.301879,2.132528,2.374963


In [28]:
# measurment on R
param_dict = {"dim":100000, "l_y":1.5, "l_m_y":0, "thr_supp":1, "l_h_r":0,  "l_h_q":0,
              "l_m":1, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 1.5, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 0, 'l_h_q': 0, 'l_m': 1, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

Correlation between R and P:  [[1.         0.89928525]
 [0.89928525 1.        ]]
The correlation matrix is:  
           P         Q         A         Y
P  1.000000  0.000310 -0.108182  0.578410
Q  0.000310  1.000000  0.001225 -0.291867
A -0.108182  0.001225  1.000000 -0.122700
Y  0.578410 -0.291867 -0.122700  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,12.943638,12.943638,10.911588,7.353967,0.590781
ACC score,100.0,100.0,81.5,81.339394,80.639394
ACC score difference,0.0,0.0,2.691576,2.855271,2.219162


In [29]:
# omission
param_dict = {"dim":100000, "l_y":1.5, "l_m_y":0, "thr_supp":1, "l_h_r":0,  "l_h_q":0,
              "l_m":0, "p_u":1, "l_r":False, "l_o":True, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 1.5, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 0, 'l_h_q': 0, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': True, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           Q         A         Y
Q  1.000000  0.001225 -0.291867
A  0.001225  1.000000 -0.122700
Y -0.291867 -0.122700  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,12.943638,12.943638,36.690088,1.163635,0.951568
ACC score,100.0,100.0,64.548485,62.624242,62.615152
ACC score difference,0.0,0.0,1.291679,2.556106,2.562163


In [30]:
# sample
param_dict = {"dim":100000, "l_y":1.5, "l_m_y":0, "thr_supp":1, "l_h_r":0,  "l_h_q":0,
              "l_m":0, "p_u":0.1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 1.5, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 0, 'l_h_q': 0, 'l_m': 0, 'p_u': 0.1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000 -0.007685 -0.001163  0.631742
Q -0.007685  1.000000 -0.002525 -0.305691
A -0.001163 -0.002525  1.000000 -0.066942
Y  0.631742 -0.305691 -0.066942  1.000000 
 The value counts is:  
 0    33493
1     3350
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,9.908602,9.908602,10.665829,0.509931,0.067498
ACC score,100.0,100.0,86.384175,86.141724,86.158254
ACC score difference,0.0,0.0,0.977419,1.555913,1.307428


In [31]:
# measurement bias on Y (P_Y as target). Performance are calculated on Y
param_dict = {"dim":100000, "l_y":1.5, "l_m_y":1.5, "thr_supp":1, "l_h_r":0,  "l_h_q":0,
              "l_m":0, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict, y_bias_meas=True)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 1.5, 'l_m_y': 1.5, 'thr_supp': 1, 'l_h_r': 0, 'l_h_q': 0, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
                R         Q         A         Y    Y_real
R       1.000000 -0.002790 -0.001676  0.603998  0.597067
Q      -0.002790  1.000000  0.001225 -0.283430 -0.307326
A      -0.001676  0.001225  1.000000 -0.244939 -0.128151
Y       0.603998 -0.283430 -0.244939  1.000000  0.890457
Y_real  0.597067 -0.307326 -0.128151  0.890457  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,24.798318,13.188308,23.100961,0.19083,0.064428
ACC score,94.193939,100.0,84.345455,83.493939,83.906061
ACC score difference,11.61001,0.0,1.073095,5.721154,4.654337


In [32]:
# representation bias
param_dict = {"dim":100000, "l_y":1.5, "l_m_y":0, "thr_supp":1, "l_h_r":0,  "l_h_q":0,
              "l_m":0, "p_u":0.1, "l_r":True, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 1.5, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 0, 'l_h_q': 0, 'l_m': 0, 'p_u': 0.1, 'l_r': True, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000 -0.005347 -0.332019  0.642284
Q -0.005347  1.000000  0.001442 -0.291786
A -0.332019  0.001442  1.000000 -0.277194
Y  0.642284 -0.291786 -0.277194  1.000000 
 The value counts is:  
 0    33493
1     3350
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Can t mitigate with CDP.
Report output. 

problem in:  Demographic parity difference
problem in:  Demographic parity ratio
problem in:  Equalized odds ratio
problem in:  Equalized odds difference
problem in:  ACC score
problem in:  F1 score
problem in:  Precision
problem in:  Recall
problem in:  ACC score difference
problem in:  F1 score difference
problem in:  Flip
problem in:  Flip
problem in:  Mutual info

,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,48.612122,48.612122,45.575221,46.720815,1.544775
ACC score,100.0,100.0,86.676218,86.836015,82.808023
ACC score difference,0.0,0.0,13.122972,12.947193,27.355793


## S5 - historical bias on R and Y

In [33]:
# no additional bias
param_dict = {"dim":100000, "l_y":1.5, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":0,
              "l_m":0, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 1.5, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': 0, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000 -0.002959 -0.174944  0.637131
Q -0.002959  1.000000  0.001225 -0.283430
A -0.174944  0.001225  1.000000 -0.244939
Y  0.637131 -0.283430 -0.244939  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,24.798318,24.798318,25.064487,13.131121,0.129787
ACC score,100.0,100.0,86.860606,86.00303,83.063636
ACC score difference,0.0,0.0,3.956293,3.780848,3.660704


In [34]:
# measurment on R
param_dict = {"dim":100000, "l_y":1.5, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":0,
              "l_m":1, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 1.5, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': 0, 'l_m': 1, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

Correlation between R and P:  [[1.         0.90538424]
 [0.90538424 1.        ]]
The correlation matrix is:  
           P         Q         A         Y
P  1.000000  0.000113 -0.259059  0.590810
Q  0.000113  1.000000  0.001225 -0.283430
A -0.259059  0.001225  1.000000 -0.244939
Y  0.590810 -0.283430 -0.244939  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,24.798318,24.798318,27.027517,18.772584,0.63316
ACC score,100.0,100.0,82.030303,81.642424,78.660606
ACC score difference,0.0,0.0,5.073201,5.036979,4.589578


In [35]:
# omission
param_dict = {"dim":100000, "l_y":1.5, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":0,
              "l_m":0, "p_u":1, "l_r":False, "l_o":True, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 1.5, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': 0, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': True, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           Q         A         Y
Q  1.000000  0.001225 -0.283430
A  0.001225  1.000000 -0.244939
Y -0.283430 -0.244939  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,24.798318,24.798318,36.690088,1.163635,0.951568
ACC score,100.0,100.0,66.421212,62.054545,62.045455
ACC score difference,0.0,0.0,4.660695,4.07105,4.077108


In [36]:
# sample
param_dict = {"dim":100000, "l_y":1.5, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":0,
              "l_m":0, "p_u":0.1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 1.5, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': 0, 'l_m': 0, 'p_u': 0.1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000 -0.007392 -0.101782  0.633864
Q -0.007392  1.000000 -0.002525 -0.301842
A -0.101782 -0.002525  1.000000 -0.124015
Y  0.633864 -0.301842 -0.124015  1.000000 
 The value counts is:  
 0    33493
1     3350
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,20.642083,20.642083,19.156915,10.290136,1.328073
ACC score,100.0,100.0,86.554992,86.334582,85.557637
ACC score difference,0.0,0.0,4.189555,1.431975,6.513473


In [37]:
# measurement bias on Y (P_Y as target). Performance are calculated on Y
param_dict = {"dim":100000, "l_y":1.5, "l_m_y":1.5, "thr_supp":1, "l_h_r":1.5,  "l_h_q":0,
              "l_m":0, "p_u":1, "l_r":False, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict, y_bias_meas=True)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 1.5, 'l_m_y': 1.5, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': 0, 'l_m': 0, 'p_u': 1, 'l_r': False, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
                R         Q         A         Y    Y_real
R       1.000000 -0.002959 -0.174944  0.619543  0.611526
Q      -0.002959  1.000000  0.001225 -0.270694 -0.293878
A      -0.174944  0.001225  1.000000 -0.359151 -0.255519
Y       0.619543 -0.270694 -0.359151  1.000000  0.902292
Y_real  0.611526 -0.293878 -0.255519  0.902292  1.000000 
 The value counts is:  
 1    33506
0    33494
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Report output. 

problem in:  Flip
problem in:  Flip


,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,36.822925,26.24909,37.392013,13.258432,0.105889
ACC score,94.712121,100.0,84.609091,84.257576,81.181818
ACC score difference,10.573835,0.0,0.097434,6.605725,11.255328


In [38]:
# representation bias
param_dict = {"dim":100000, "l_y":1.5, "l_m_y":0, "thr_supp":1, "l_h_r":1.5,  "l_h_q":0,
              "l_m":0, "p_u":0.1, "l_r":True, "l_o":False, "l_y_b":0,
              "l_q":2, "sy":2, "l_r_q":0}
dtf_res = pipeline(param_dict)
# Select only DP diff, Accuracy, Accuracy difference for the models Unmitigated, FTU and DP postprocessed
dtf_res.loc[['Demographic parity difference', 'ACC score', 'ACC score difference'], ['Y', 'Y True', 'Unmitigated', 'FTU', 'DP']]*100

The parameters for data generetion are:  {'dim': 100000, 'l_y': 1.5, 'l_m_y': 0, 'thr_supp': 1, 'l_h_r': 1.5, 'l_h_q': 0, 'l_m': 0, 'p_u': 0.1, 'l_r': True, 'l_o': False, 'l_y_b': 0, 'l_q': 2, 'sy': 2, 'l_r_q': 0} 

Start creation dataset. 

The correlation matrix is:  
           R         Q         A         Y
R  1.000000 -0.005293 -0.416227  0.644387
Q -0.005293  1.000000  0.001442 -0.290510
A -0.416227  0.001442  1.000000 -0.289102
Y  0.644387 -0.290510 -0.289102  1.000000 
 The value counts is:  
 0    33493
1     3350
Name: A, dtype: int64 

Fitting models. 

Mitigate models. 

Can t mitigate with CDP.
Report output. 

problem in:  Demographic parity difference
problem in:  Demographic parity ratio
problem in:  Equalized odds ratio
problem in:  Equalized odds difference
problem in:  ACC score
problem in:  F1 score
problem in:  Precision
problem in:  Recall
problem in:  ACC score difference
problem in:  F1 score difference
problem in:  Flip
problem in:  Flip
problem in:  Mutual in

,Y,Y True,Unmitigated,FTU,DP
Demographic parity difference,50.721366,50.721366,46.993575,48.430113,1.054034
ACC score,100.0,100.0,86.621115,86.791933,82.510469
ACC score difference,0.0,0.0,14.116929,13.929027,29.761844
